<a href="https://colab.research.google.com/github/DeokhuiHan/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_5%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오픈소스 기반 데이터 분석 5강 - 데이터 저장

## 5-1 CSV 형식 저장

In [55]:
import requests

# 1. API KEY 설정 (사용자가 발급받은 키로 변경 필요)
API_KEY = "554a6d55756a75723438656b43596f"

# 2. 연도 및 월 범위 설정
start_year = 2015
end_year = 2024

# 3. 월별 데이터 수집 함수
def fetch_energy_data(year, month):
    base_url = f"http://openapi.seoul.go.kr:8088/554a6d55756a75723438656b43596f/json/energyUseDataSummaryInfo/1/5/{year}/{month}"

    try:
        response = requests.get(base_url)
        if response.status_code == 200:
            data = response.json()
            #print(f"[{year}년 {month}월] 데이터 수집 성공")
            return data
        else:
            print(f"[{year}년 {month}월] API 호출 실패 (상태 코드: {response.status_code})")
    except Exception as e:
        print(f"[{year}년 {month}월] 연결 오류 발생: {str(e)}")

# 4. 전체 기간 데이터 수집 실행
total_count = 0
failed_count = 0

print("=== 에너지 사용량 데이터 수집 시작 ===")
for year in range(start_year, end_year + 1):
    for month in range(12):  # 0~11 (1월~12월)
        result = fetch_energy_data(year, month)
        if result:
            total_count += 1
        else:
            failed_count += 1

print(f"\n=== 수집 결과 ===")
print(f"총 성공: {total_count}건")
print(f"실패: {failed_count}건")


=== 에너지 사용량 데이터 수집 시작 ===

=== 수집 결과 ===
총 성공: 120건
실패: 0건


In [51]:
import requests
import pandas as pd

api_key = "554a6d55756a75723438656b43596f"
start_year = 2015
end_year = 2024

def fetch_energy_data(api_key, start_year, end_year):
    base_url = "http://openapi.seoul.go.kr:8088/554a6d55756a75723438656b43596f/json/energyUseDataSummaryInfo/1/5/{year}/{month}"
    all_data = []
    dates_to_fetch = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            dates_to_fetch.append((year, month))

    for year, month in tqdm(dates_to_fetch, desc="데이터 수집 중"):
        month_str = str(month).zfill(2)
        url = base_url.format(key=api_key, year=year, month=month_str)

        try:
            response = requests.get(url)
            response.raise_for_status()
            json_data = response.json()

            if 'energyUseDataSummaryInfo' in json_data and 'row' in json_data['energyUseDataSummaryInfo']:
                row_data = json_data['energyUseDataSummaryInfo']['row'][0]
                row_data['BASE_YEAR'] = str(year)
                row_data['BASE_MONTH'] = month_str
                all_data.append(row_data)
            else:
                print(f"[{year}-{month_str}] 데이터 구조 오류 또는 데이터 없음")

        except requests.exceptions.RequestException as e:
            print(f"[{year}-{month_str}] API 호출 오류: {e}")

        except requests.exceptions.JSONDecodeError as e:
            print(f"[{year}-{month_str}] JSON 디코딩 오류: {e}")

    return all_data

collected_data = fetch_energy_data(SERVICE_KEY, START_YEAR, END_YEAR)

print("\n--- 수집 완료된 데이터 (일부) ---")
print(collected_data[:5])
print(f"총 {len(collected_data)}개월 분 데이터 수집 완료.")

df = pd.DataFrame(collected_data)
df

데이터 수집 중: 100%|██████████| 120/120 [00:23<00:00,  5.17it/s]



--- 수집 완료된 데이터 (일부) ---
[{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0', 'BASE_YEAR': '2015', 'BASE_MONTH': '01'}, {'YEAR': '2015', 'MON': '02', 'MM_TYPE': '개인', 'CNT': '774620', 'EUS': '189974230', 'EUS1': '193611430', 'EUS2': '200055533', 'ECO2_1': '-6859251.5', 'ECO2_2': '-2908322.636', 'GUS': '56487358', 'GUS1': '59353536', 'GUS2': '66191173', 'GCO2_1': '-6284996.5', 'GCO2_2': '-14078392.16', 'WUS': '12656888.218', 'WUS1': '12713146.172', 'WUS2': '12948410.081', 'WCO2_1': '-173

,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE,BASE_YEAR,BASE_MONTH
0,2015,01,개인,767791,193784708,194781915,204969429,-6090964,-2582568.736,59133720,...,8179.308,2715.530256,22740838.937,23400055.303,27090493.875,-2504435.652,-33660084.213069,2015-06-04 17:03:55.0,2015,01
1,2015,02,개인,774620,189974230,193611430,200055533,-6859251.5,-2908322.636,56487358,...,-173889.9085,-57731.449622,18793320.213,19032221.757,21414367.325,-1429974.328,-17311712.76832,2015-06-04 16:59:30.0,2015,02
2,2015,03,학교,3903,145577747,140333207,148915260,673170,285424.08,7601585,...,-137612.4,-45687.3168,48,34,41,10.5,41.811,2015-06-12 04:00:01.0,2015,03
3,2015,04,학교,4021,134199179,130280899,148944597,-5413569,-2295353.256,5544226,...,-106328.8,-35301.1616,9,8,16,-3,-11.946,2015-07-05 04:00:00.0,2015,04
4,2015,05,개인,810422,177289769,181972266,188230004,-7811366,-3312019.184,22060892,...,-113657.49,-37734.28668,1320895.801,1558697.648,1709192.194,-313049.12,-5190153.166471,2015-08-05 04:00:01.0,2015,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2024,08,종교단체,847,9717678,9362542,8951131,560841.5,560841.5,176486,...,1437.5,4312.5,0,0,0,0,0,2024-11-01 04:04:00.0,2024,08
116,2024,09,기업,19450,742787183,713886345,683362396,44162812.5,44162812.5,22085380,...,214271.2,642813.6,0,0,0,0,0,2024-12-01 04:02:35.0,2024,09
117,2024,10,개인,1249966,282268119,264132620,263389786,18506916,14379201.216,8233421,...,-148610.9,-189025.694,2695491.396,3326775.236,4339830.387,-1137811.4155,-1573249.33785276,2025-01-01 04:02:38.0,2024,10
118,2024,11,공공기관,2456,141281942,145555106,141150651,-2070936.5,-2070936.5,1070872,...,115454.4,346363.2,0,0,0,0,0,2025-02-01 04:02:31.0,2024,11


In [ ]:


data = {
    'student_id': [101, 102, 103, 104, 105],
    'database_score': [85, 76, 92, 63, 88],
    'cloudcomputing_score': [78, 82, 95, 70, 84],
    'python_score': [92, 78, 85, 75, 91],
    'watch_rate': [0.95, 0.87, 0.99, 0.80, 0.93]
}
## DataFrame 생성

## CSV 형식 저장


## 5-2 JSON 형식 저장

In [5]:
### 라이브러리 임포트
import json
import pandas as pd

data = {
    "이름": "홍길동",
    "나이": 25,
    "거주지": "서울",
    "관심사": ["프로그래밍", "데이터 분석", "여행"]
}

## json.dump를 이용한 저장
with open('output.jason', mode='w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
## DataFrame을 이용한 저장
df= pd.DataFrame([data])
df.to_json('output_df.json', orient='columns', indent=4, force_ascii=False)

## 5-3 Pandas Dataframe 데이터 추출

In [11]:
import pandas as pd

data = {
    "이름": ["김철수", "이영희", "박민수", "최지훈", "정소희"],
    "학년": [1, 2, 3, 4, 2],
    "학점": [4.2, 3.8, 4.5, 3.9, 3.5],
    "학과": ["컴퓨터학", "경영학", "농학", "교육학", "영어영문학"],
    "동아리": ["프로그래밍", "독서토론", "로봇공학", "봉사활동", "음악감상"]
}
df = pd.DataFrame(data)
df
## DataFrame 인덱스 출력
print(df.index)
## DataFrame 컬럼 출력
print(df.columns)
## DataFrame 행 출력
print(df.values.tolist())
print(df.values)
## DataFrame 값 출력
print(df.values.flatten())

RangeIndex(start=0, stop=5, step=1)
Index(['이름', '학년', '학점', '학과', '동아리'], dtype='object')
[['김철수', 1, 4.2, '컴퓨터학', '프로그래밍'], ['이영희', 2, 3.8, '경영학', '독서토론'], ['박민수', 3, 4.5, '농학', '로봇공학'], ['최지훈', 4, 3.9, '교육학', '봉사활동'], ['정소희', 2, 3.5, '영어영문학', '음악감상']]
[['김철수' 1 4.2 '컴퓨터학' '프로그래밍']
 ['이영희' 2 3.8 '경영학' '독서토론']
 ['박민수' 3 4.5 '농학' '로봇공학']
 ['최지훈' 4 3.9 '교육학' '봉사활동']
 ['정소희' 2 3.5 '영어영문학' '음악감상']]
['김철수' 1 4.2 '컴퓨터학' '프로그래밍' '이영희' 2 3.8 '경영학' '독서토론' '박민수' 3 4.5 '농학'
 '로봇공학' '최지훈' 4 3.9 '교육학' '봉사활동' '정소희' 2 3.5 '영어영문학' '음악감상']


## 5-4 DataFrame 생성 방법

In [13]:
import pandas as pd
import json

data = {
    "이름": ["김철수", "이영희", "박민수", "최지훈", "정소희"],
    "학년": [1, 2, 3, 4, 2],
    "학점": [4.2, 3.8, 4.5, 3.9, 3.5],
    "학과": ["컴퓨터공학", "경영학", "전자공학", "의학", "심리학"],
    "동아리": ["프로그래밍", "독서토론", "로봇공학", "봉사활동", "음악감상"]
}

## 딕셔너리, csv, json으로 부터 DataFrame 생성
df1 = pd.DataFrame.from_dict(data)
df2 = pd.read_csv('students.csv')
df3 = pd.read_json('students.json')

## 세가지 방법 결과 비교
print(df1.shape)
print(df2.shape)
print(df3.shape)

(5, 5)
(5, 5)
(5, 5)


## 5-5 DataFrame 저장 방법

In [15]:
import pandas as pd

data = {
    "이름": ["김철수", "이영희", "박민수", "최지훈", "정소희"],
    "학년": [1, 2, 3, 4, 2],
    "학점": [4.2, 3.8, 4.5, 3.9, 3.5],
    "학과": ["컴퓨터공학", "경영학", "전자공학", "의학", "심리학"],
    "동아리": ["프로그래밍", "독서토론", "로봇공학", "봉사활동", "음악감상"]
}

## Dataframe 생성
df1 = pd.DataFrame(data)

## CSV, JSON 형식으로 저장
df.to_csv('students_score.csv')
df.to_json('students_score.csv')

## HTML 형식으로 저장
html_table = df.to_html()
with open('stedents_scores.html', 'w') as f:
    f.write(html_table)


## 5-6 실습 시나리오

## 데이터 수집

In [24]:
import requests
url = "http://openapi.seoul.go.kr:8088/554a6d55756a75723438656b43596f/json/energyUseDataSummaryInfo/1/5/2015/01"
api_key = "554a6d55756a75723438656b43596f"
params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}
response = requests.get(url, params=params)
if response.status_code == 200:
  print("api 호출 성공")
  print(response.json())
else:
  print(f"API 호출 실패: {response.status_code}")

api 호출 성공
{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0'}, {'YEAR': '2015', 'MON': '01', 'MM_TYPE': '학교', 'CNT': '1382', 'EUS': '134955565', 'EUS1': '128707423', 'EUS2': '145561511', 'ECO2_1': '-2178902', 'ECO2_2': '-923854.448', 'GUS': '9107197', 'GUS1': '8410968', 'GUS2': '10745416', 'GCO2_1': '-470995', 'GCO2_2': '-1055028.8', 'WUS': '2075819.2', 'WUS1': '2097433.

In [28]:
import requests

url = "http://openapi.seoul.go.kr:8088/554a6d55756a75723438656b43596f/json/energyUseDataSummaryInfo/1/5/2015/01"
api_key = "554a6d55756a75723438656b43596"

params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

## 데이터 수집
response = requests.get(url, params=params)

## 호출 성공/실패 출력
print(response.json())
response.json()


{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0'}, {'YEAR': '2015', 'MON': '01', 'MM_TYPE': '학교', 'CNT': '1382', 'EUS': '134955565', 'EUS1': '128707423', 'EUS2': '145561511', 'ECO2_1': '-2178902', 'ECO2_2': '-923854.448', 'GUS': '9107197', 'GUS1': '8410968', 'GUS2': '10745416', 'GCO2_1': '-470995', 'GCO2_2': '-1055028.8', 'WUS': '2075819.2', 'WUS1': '2097433.8', 'WUS2'

{'energyUseDataSummaryInfo': {'list_total_count': 7,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'YEAR': '2015',
    'MON': '01',
    'MM_TYPE': '개인',
    'CNT': '767791',
    'EUS': '193784708',
    'EUS1': '194781915',
    'EUS2': '204969429',
    'ECO2_1': '-6090964',
    'ECO2_2': '-2582568.736',
    'GUS': '59133720',
    'GUS1': '57163993',
    'GUS2': '68297619',
    'GCO2_1': '-3597086',
    'GCO2_2': '-8057472.64',
    'WUS': '12819757.886',
    'WUS1': '12723680.426',
    'WUS2': '12899476.73',
    'WCO2_1': '8179.308',
    'WCO2_2': '2715.530256',
    'HUS': '22740838.937',
    'HUS1': '23400055.303',
    'HUS2': '27090493.875',
    'HCO2_1': '-2504435.652',
    'HCO2_2': '-33660084.213069',
    'REG_DATE': '2015-06-04 17:03:55.0'},
   {'YEAR': '2015',
    'MON': '01',
    'MM_TYPE': '학교',
    'CNT': '1382',
    'EUS': '134955565',
    'EUS1': '128707423',
    'EUS2': '145561511',
    'ECO2_1': '-2178902',
    'ECO2_2': '-923854.448',
    'GUS': '91

## 데이터 프레임 생성

In [37]:
import pandas as pd

## requests로부터 response, body, item 항목 읽기
data = response.json()
print(type(data))
print(len(data))
print(data)
df = pd.DataFrame(data)
df['wus']

<class 'dict'>
1
{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0'}, {'YEAR': '2015', 'MON': '01', 'MM_TYPE': '학교', 'CNT': '1382', 'EUS': '134955565', 'EUS1': '128707423', 'EUS2': '145561511', 'ECO2_1': '-2178902', 'ECO2_2': '-923854.448', 'GUS': '9107197', 'GUS1': '8410968', 'GUS2': '10745416', 'GCO2_1': '-470995', 'GCO2_2': '-1055028.8', 'WUS': '2075819.2', 'WUS1': '2

KeyError: 'wus'

## 다양항 형식으로 데이터 저장


In [ ]:
df.to_csv('air_pollution.csv', index=False, encoding='utf-8')

## JSON 형식 데이터 저장

## EXCEL 형식 데이터 저장

## HTML 형식 데이터 저장
